In [1]:
import os

import keras
from imutils import paths
import tensorflow as tf

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import imageio
import cv2
from IPython.display import Image
from sklearn.metrics.pairwise import cosine_similarity

2024-10-21 13:34:43.835796: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-21 13:34:43.840693: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-21 13:34:43.883260: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-21 13:34:43.926389: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 13:34:43.964691: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
# Load the pre-trained ResNet50 model, excluding the top layer
model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, pooling='avg')

I0000 00:00:1729497886.545523    5557 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-21 13:34:46.545916: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
def preprocess_frame(frame):
    frame = cv2.resize(frame, (224, 224))  # Resize frame to 224x224
    frame = tf.keras.applications.resnet50.preprocess_input(frame)  # Preprocess for ResNet50
    return frame

In [4]:
def extract_features_from_video(video_path, model):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        return None  # Return None if the video file can't be opened

    features = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess the frame and add batch dimension
        frame = preprocess_frame(frame)
        frame = np.expand_dims(frame, axis=0)

        # Extract features using ResNet50
        feature = model.predict(frame)
        features.append(feature.flatten())

    cap.release()

    # Check if features list is empty
    if not features:
        print(f"Warning: No frames extracted from video file {video_path}")
        return None  # Return None if no frames were processed

    # Average features across all frames
    video_features = np.mean(features, axis=0)
    return video_features


In [5]:
def find_similar_videos(input_video_path, dataset_video_paths, model, top_k=5):
    # Extract features from the input video
    input_features = extract_features_from_video(input_video_path, model)
    if input_features is None:
        print(f"Error: Unable to extract features from the input video {input_video_path}")
        return [], []  # Return empty lists if input features can't be extracted

    # Extract features from each video in the dataset
    dataset_features = []
    valid_video_paths = []
    for video_path in dataset_video_paths:
        features = extract_features_from_video(video_path, model)
        if features is not None:
            dataset_features.append(features)
            valid_video_paths.append(video_path)

    # Check if any valid features were extracted
    if not dataset_features:
        print("Error: No valid features extracted from the dataset videos")
        return [], []

    # Compute cosine similarity between input video and dataset videos
    similarities = cosine_similarity([input_features], dataset_features).flatten()

    # Get the indices of the top_k most similar videos
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    top_similar_videos = [valid_video_paths[i] for i in top_indices]

    return top_similar_videos, top_indices


In [6]:
# Read video paths and labels from the CSV file
def load_video_paths_and_labels_from_csv(csv_file_path):
    df = pd.read_csv(csv_file_path)
    return df['Video File'].tolist(), df['Label'].tolist()

In [8]:
def calculate_precision_recall(input_label, dataset_labels, top_indices):
    # Count relevant videos in top retrieved videos
    relevant_retrieved = sum(1 for i in top_indices if dataset_labels[i] == input_label)
    total_relevant = dataset_labels.count(input_label)
    total_retrieved = len(top_indices)

    # Calculate precision and recall
    precision = relevant_retrieved / total_retrieved if total_retrieved > 0 else 0
    recall = relevant_retrieved / total_relevant if total_relevant > 0 else 0

    return precision, recall

In [9]:
# Example usage
csv_file_path = 'UCF101_labeled_data.csv'  # Path to the CSV file containing video paths and labels
input_video_path = 'v_BreastStroke_g17_c01.avi'  # Path to the input video
input_video_label = 'BreastStroke'  # Label of the input video
top_k = 5

# Load dataset video paths and labels from the CSV file
dataset_video_paths, dataset_labels = load_video_paths_and_labels_from_csv(csv_file_path)

# Find the top 5 similar videos
similar_videos, top_indices = find_similar_videos(input_video_path, dataset_video_paths, model, top_k)
print("Top 5 similar videos:", similar_videos)

# Calculate precision and recall
precision, recall = calculate_precision_recall(input_video_label, dataset_labels, top_indices)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━